# Creates and simulates a circuit equivalent to a Bell inequality test

Referance:
https://cirq.readthedocs.io/en/stable/examples.html#bell-inequality

In [2]:
import numpy as np
import cirq

In [3]:
alice = cirq.GridQubit(0, 0)
bob = cirq.GridQubit(1, 0)
alice_referee = cirq.GridQubit(0, 1)
bob_referee = cirq.GridQubit(1, 1)

In [4]:
circuit = cirq.Circuit()

In [5]:
# Prepare shared entangled state.
circuit.append([
    cirq.H(alice),
    cirq.CNOT(alice, bob),
    cirq.X(alice)**-0.25,
])

In [6]:
# Referees flip coins.
circuit.append([
    cirq.H(alice_referee),
    cirq.H(bob_referee),
], strategy=cirq.InsertStrategy.NEW_THEN_INLINE)

In [7]:
# Players do a sqrt(X) based on their referee's coin.
circuit.append([
    cirq.CNOT(alice_referee, alice)**0.5,
    cirq.CNOT(bob_referee, bob)**0.5,
], strategy=cirq.InsertStrategy.NEW_THEN_INLINE)

In [8]:
# Then results are recorded.
circuit.append([
    cirq.measure(alice, key='a'),
    cirq.measure(bob, key='b'),
    cirq.measure(alice_referee, key='x'),
    cirq.measure(bob_referee, key='y'),
], strategy=cirq.InsertStrategy.NEW_THEN_INLINE)

In [9]:
# Show circuit.
print(circuit)

(0, 0): ───H───@───X^-0.25───────X───────M('a')───
               │                 │
(0, 1): ───────┼─────────────H───@^0.5───M('x')───
               │
(1, 0): ───────X─────────────────X───────M('b')───
                                 │
(1, 1): ─────────────────────H───@^0.5───M('y')───


In [10]:
# Run simulations.
repetitions = 10000
print('Simulating {} repetitions...'.format(repetitions))
result = cirq.Simulator().run(program=circuit, repetitions=repetitions)

Simulating 10000 repetitions...


In [11]:
# Collect results.
a = np.array(result.measurements['a'][:, 0])
b = np.array(result.measurements['b'][:, 0])
x = np.array(result.measurements['x'][:, 0])
y = np.array(result.measurements['y'][:, 0])
outcomes = a ^ b == x & y
win_percent = len([e for e in outcomes if e]) * 100 / repetitions

In [12]:
def bitstring(bits):
    return ''.join('1' if e else '_' for e in bits)

In [14]:
# Print data.
print('a:', bitstring(a[:100]))
print('b:', bitstring(b[:100]))
print('x:', bitstring(x[:100]))
print('y:', bitstring(y[:100]))
print('(a XOR b) == (x AND y):\n  ', bitstring(outcomes[:100]))
print('Win rate: {}%'.format(win_percent))

a: 1_111____1111111__________11_11_1_111_1111_11_1_11_11__1_11111_11____11_1__11____1_111_11______1_1__
b: ___1_1___111_11___1__1__1_11___11_1_1_11_11111_1_1__1__1_1_1_1_1___1__1_1_1111_____111111________1__
x: 1__11___11___1_1_11__1__1_11__111_11_1_111_1_1111_1____1111_1___11_1___1__1_11___1____1_______111__1
y: 11__1__1___11__1__1__1_11_1__11____1__1_1_1_1_111111____1___1__11__11_1___1____1111_1111_____1_1____
(a XOR b) == (x AND y):
   11_11_111111_1111111111111_11_1_1111111111_11_1111__1111_1_1111111111_1111111_1111111111111111111111
Win rate: 85.75%


In [17]:
# Theoretical win rate
np.cos(np.pi/8)**2

0.8535533905932737